-  for each line in smetana outputs, generate a triple in a consumer resource model consisting of donor organism, exchange metabolite, and recipient organism
- convert triple into reactions in antimony as save as SBML
- load SBML into MIRA
- load MIRA into pyCIEMSS (specify priors)
- simulate the model in pyCIEMSS (specify initial conditions)

### for each line in smetana outputs, generate a triple in a consumer resource model consisting of donor organism, exchange metabolite, and recipient organism

In [16]:
import pandas as pd
import tellurium as te
import antimony as at
import simplesbml 
import mira
import libsbml
import numpy as np


In [17]:
smetana_file = "../data/smetana_results/smetana_results_gurobi_victor_detailed.tsv"

smetana_df = pd.read_csv(smetana_file, sep='\t')

smetana_df

community   medium        receiver           donor    compound       scs  \
0         all  minimal  updated_csc009  updated_csc031  M_arg__L_e  0.333333   
1         all  minimal  updated_csc009  updated_csc031     M_gua_e  0.333333   
2         all  minimal  updated_csc009  updated_csc031     M_h2s_e  0.333333   
3         all  minimal  updated_csc009  updated_csc031  M_hom__L_e  0.333333   
4         all  minimal  updated_csc009  updated_csc031     M_nh4_e  0.333333   
..        ...      ...             ...             ...         ...       ...   
104       all  minimal          csc043  updated_csc040       M_s_e  1.000000   
105       all  minimal          csc043  updated_csc052  M_arg__L_e  0.333333   
106       all  minimal          csc043  updated_csc052     M_h2s_e  0.333333   
107       all  minimal          csc043  updated_csc052  M_lys__L_e  0.333333   
108       all  minimal          csc043  updated_csc052       M_s_e  0.333333   

      mus  mps   smetana  
0    0.08    1  0.026667  
1    0.10    1  0.033333  
2    0.33    1  0.110000  
3    0.07    1  0.023333  
4    0.09    1  0.030000  
..    ...  ...       ...  
104  0.22    1  0.220000  
105  0.22    1  0.073333  
106  0.78    1  0.260000  
107  0.46    1  0.153333  
108  0.22    1  0.073333  

[109 rows x 9 columns]

In [50]:
np.random.seed(0)

reaction_list = []
parameter_list = []
initial_values_set = set()


for index, row in smetana_df.iterrows():
    donor_reaction = f'\t{row.donor} -> {row.compound}; k_d{index}*{row.donor}'
    receiver_reaction = f'\t{row.compound} -> {row.receiver}; k_r{index}*{row.compound}'
    reaction_list.extend([donor_reaction,receiver_reaction])
    parameter_list.extend([f'\tk_d{index}={row.scs}',f'\tk_r{index}={row.mus}'])
    initial_values_set.add(row.donor)
    initial_values_set.add(row.compound)
    initial_values_set.add(row.receiver)

initial_values_string = "\n".join([f"\t{name}={np.random.uniform(0,1)}" for name in initial_values_set])
species_names_string = "\n".join([f'\t{species} is "{species}"' for species in initial_values_set])

reaction_string = "\n".join(reaction_list)
parameter_string = "\n".join(parameter_list)

antimony_file = f"""
model smetana
{reaction_string}
{parameter_string}
{initial_values_string}
{species_names_string}
end
"""

print(antimony_file)


model smetana
	updated_csc031 -> M_arg__L_e; k_d0*updated_csc031
	M_arg__L_e -> updated_csc009; k_r0*M_arg__L_e
	updated_csc031 -> M_gua_e; k_d1*updated_csc031
	M_gua_e -> updated_csc009; k_r1*M_gua_e
	updated_csc031 -> M_h2s_e; k_d2*updated_csc031
	M_h2s_e -> updated_csc009; k_r2*M_h2s_e
	updated_csc031 -> M_hom__L_e; k_d3*updated_csc031
	M_hom__L_e -> updated_csc009; k_r3*M_hom__L_e
	updated_csc031 -> M_nh4_e; k_d4*updated_csc031
	M_nh4_e -> updated_csc009; k_r4*M_nh4_e
	updated_csc031 -> M_orn_e; k_d5*updated_csc031
	M_orn_e -> updated_csc009; k_r5*M_orn_e
	updated_csc040 -> M_4abut_e; k_d6*updated_csc040
	M_4abut_e -> updated_csc009; k_r6*M_4abut_e
	updated_csc040 -> M_arg__L_e; k_d7*updated_csc040
	M_arg__L_e -> updated_csc009; k_r7*M_arg__L_e
	updated_csc040 -> M_asp__L_e; k_d8*updated_csc040
	M_asp__L_e -> updated_csc009; k_r8*M_asp__L_e
	updated_csc040 -> M_fe3_e; k_d9*updated_csc040
	M_fe3_e -> updated_csc009; k_r9*M_fe3_e
	updated_csc040 -> M_fe3pyovd_kt_e; k_d10*updated_csc

### convert triple into reactions in antimony as save as SBML

In [56]:
smbl_file = 'smetana_carbstor_consumer_resource.xml'
rr_model = te.loada(antimony_file)
rr_model.exportToSBML(smbl_file)

# simulate and plot
rr_model.simulate(0, 1000, 2000)
rr_model.plot()

In [58]:
import libsbml
corrected_smbl_file = 'smetana_carbstor_consumer_resource_corrected.xml'

d = libsbml.readSBMLFromFile(smbl_file)
m = d.getModel()
# for initial in m.getListOfInitialAssignments():
#     print(initial)

for species in m.getListOfSpecies():
    species.name = species.id
    species.initial_concentration = species.initial_amount
    species.has_only_substance_units = False

libsbml.writeSBMLToFile(d,corrected_smbl_file)

1

### load SBML into MIRA


In [59]:
from mira.sources.sbml import template_model_from_sbml_file



In [76]:
smetana_model = template_model_from_sbml_file(corrected_smbl_file)

smetana_model.initials


{'updated_csc031': Initial(concept=Concept(name='updated_csc031', display_name='updated_csc031', description=None, identifiers={}, context={}, units=None), expression=0.925596638292661),
 'M_arg__L_e': Initial(concept=Concept(name='M_arg__L_e', display_name='M_arg__L_e', description=None, identifiers={}, context={}, units=None), expression=0.791725038082665),
 'updated_csc009': Initial(concept=Concept(name='updated_csc009', display_name='updated_csc009', description=None, identifiers={}, context={}, units=None), expression=0.639921021327524),
 'M_gua_e': Initial(concept=Concept(name='M_gua_e', display_name='M_gua_e', description=None, identifiers={}, context={}, units=None), expression=0.568044561093932),
 'M_h2s_e': Initial(concept=Concept(name='M_h2s_e', display_name='M_h2s_e', description=None, identifiers={}, context={}, units=None), expression=0.715189366372419),
 'M_hom__L_e': Initial(concept=Concept(name='M_hom__L_e', display_name='M_hom__L_e', description=None, identifiers={}, 

In [77]:
from mira.metamodel import Distribution


dist = Distribution(
                    type= "Uniform1",
                    parameters= {"minimum": 0.0,"maximum": 1.0 }     
                     )

for name,parameter in smetana_model.parameters.items():
    if name != "default_compartment":
        parameter.distribution = dist

parameter

Parameter(name='default_compartment', display_name=None, description='', identifiers={}, context={}, units=None, value=1.0, distribution=None)

In [11]:
# initial conditions 


### load MIRA into pyCIEMSS (specify priors)
### simulate the model in pyCIEMSS (specify initial conditions)

see: https://github.com/ciemss/pyciemss/blob/main/docs/source/interfaces.ipynb


In [61]:
import os
import pyciemss
import torch
import pandas as pd
import numpy as np
from typing import Dict, List, Callable

import pyciemss.visuals.plots as plots
import pyciemss.visuals.vega as vega
import pyciemss.visuals.trajectories as trajectories

from pyciemss.integration_utils.intervention_builder import (
    param_value_objective,
    start_time_objective,
    start_time_param_value_objective,
)

smoke_test = "CI" in os.environ



In [78]:


# load model
# load data

# 

# set parameters
start_time = 0.0
end_time = 100.0
logging_step_size = 10.0
num_samples = 1

In [80]:
result1 = pyciemss.sample(smetana_model, end_time, logging_step_size, num_samples, start_time=start_time)
display(result1['data'].head())



timepoint_id  sample_id timepoint_unknown  persistent_k_d0_param  \
0             0          0               0.0               0.109258   
1             1          0              10.0               0.109258   
2             2          0              20.0               0.109258   
3             3          0              30.0               0.109258   
4             4          0              40.0               0.109258   

   persistent_k_r0_param  persistent_k_d1_param  persistent_k_r1_param  \
0               0.537517               0.892519               0.732601   
1               0.537517               0.892519               0.732601   
2               0.537517               0.892519               0.732601   
3               0.537517               0.892519               0.732601   
4               0.537517               0.892519               0.732601   

   persistent_k_d2_param  persistent_k_r2_param  persistent_k_d3_param  ...  \
0               0.591642               0.189439               0.866161  ...   
1               0.591642               0.189439               0.866161  ...   
2               0.591642               0.189439               0.866161  ...   
3               0.591642               0.189439               0.866161  ...   
4               0.591642               0.189439               0.866161  ...   

   csc043_state  M_xylan4_e_state  M_glyald_e_state  M_glyc_e_state  \
0      0.461479          0.978618          0.870012        0.087129   
1      0.407688          0.215156          0.863196        0.601776   
2      0.406272          0.214269          0.860323        0.599925   
3      0.406200          0.214225          0.860186        0.599833   
4      0.406196          0.214222          0.860178        0.599827   

   M_rib__D_e_state  M_lys__L_e_state  M_o2_e_state  M_ura_e_state  \
0          0.020218          0.799159      0.891773       0.963663   
1          0.494157          0.699344      1.194294       0.885769   
2          0.492808          0.697277      1.190029       0.877605   
3          0.492754          0.697174      1.189798       0.877092   
4          0.492751          0.697168      1.189783       0.877062   

   M_xyl__D_e_state  M_ppap_e_state  
0          0.437587        0.602763  
1          1.005388        0.979565  
2          1.043010        0.975040  
3          1.045780        0.974786  
4          1.045995        0.974769  

[5 rows x 253 columns]

In [81]:
# Plot results for all states
schema = plots.trajectories(result1["data"], keep=".*_state")
plots.save_schema(schema, "_schema.json")
plots.ipy_display(schema, dpi=150)